In [2]:

import requests
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_timestamp
from pyspark.sql import Row
from datetime import datetime

# API URL
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "ids": "bitcoin,ethereum,solana,cardano,dogecoin",
    "order": "market_cap_desc",
    "per_page": 5,
    "page": 1,
    "sparkline": "false"
}

# Fetch Data
# response = requests.get(url, params=params)
# data = response.json()

# Fetch Data and Log error
try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print("❌ API Error:", e)

    # Prepare error details
    error_details = [{
        "error_message": str(e),
        "source": "CoinGecko API",
        "timestamp": datetime.utcnow().isoformat()
    }]
    
    # Convert to Spark DataFrame
    error_df = spark.createDataFrame([Row(**row) for row in error_details])

    # Write or append to Delta table
    error_df.write.mode("append").format("delta").saveAsTable("api_error_logs")

    print("🚨 Error logged to 'api_error_logs' table.")


# Convert to Pandas DataFrame
df = pd.json_normalize(data)

# Add ingestion timestamp
df['ingestion_timestamp'] = datetime.utcnow()

# Select and rename columns
df = df[[
    'id', 'symbol', 'name', 'current_price', 'market_cap',
    'price_change_24h', 'price_change_percentage_24h',
    'last_updated', 'ingestion_timestamp'
]]
df.columns = [
    'coin_id', 'symbol', 'name', 'current_price', 'market_cap',
    'price_change_24h', 'price_change_percentage_24h',
    'last_updated', 'ingestion_timestamp'
]

# Convert to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Write to Delta table (append mode)
target_table = "coin_price_data"
spark_df.write.mode("append").format("delta").saveAsTable(target_table)

print("✅ Data fetched and saved to Lakehouse table.")


StatementMeta(, d7959c11-b53e-4bfa-a46a-f73520dab85f, 4, Finished, Available, Finished)

✅ Data fetched and saved to Lakehouse table.
